In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import warnings
warnings.filterwarnings("ignore")
import pickle
import sys
main_path = r'../..'
sys.path.append(main_path)
from BayDS import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
main_folder = r'f:\my\Prog\kaggle\Stacking_data'
model_folder = r'f:\my\Prog\kaggle\Snapshots\Stacking\06.09'

In [5]:
train = pd.read_pickle(f'../../Data/train_09457_with_additions.pkl')
test = pd.read_pickle(f'../../Data/test_09457_with_additions.pkl')


In [6]:
train.index

Int64Index([2987000, 2987001, 2987002, 2987003, 2987004, 2987005, 2987006,
            2987007, 2987008, 2987009,
            ...
            3577530, 3577531, 3577532, 3577533, 3577534, 3577535, 3577536,
            3577537, 3577538, 3577539],
           dtype='int64', name='TransactionID', length=590540)

In [7]:
L = os.listdir(main_folder)
oof = pd.DataFrame(index=train.index)
prediction = pd.DataFrame(index=test.index)
for experiment_name in L:
    experiment_folder = os.path.join(main_folder,experiment_name)
    if not os.path.exists(f'{experiment_folder}/results_dict.pkl'):
        continue
    results_dict = pd.read_pickle(f'{experiment_folder}/results_dict.pkl')
    print(experiment_name, sum(results_dict['scores'])/len(results_dict['scores']))
    oof[experiment_name] = results_dict['oof'][:,0]
    prediction[experiment_name] = results_dict['prediction'][:,0]
y = train.isFraud
    

keras-1 0.9424890970079728
keras-3 0.9335847806217249
lightgbm-0000 0.9214079716576545
lightgbm-0001 0.9307534121273541
lightgbm-0002 0.9057393914936828
lightgbm-0003 0.9094343223387966
lightgbm-0004 0.922207412827418
lightgbm-0005 0.9326247096098855
lightgbm-0006 0.919660639645335
lightgbm-0007 0.9193543696157349
lightgbm-0008 0.9343375118661627
lightgbm-0009 0.9195492820489312
lightgbm-0010 0.930241049291918
lightgbm-0011 0.9320579425282656
lightgbm-0012 0.9088114853323285
logreg-0000 0.7427794669324287
logreg-0001 0.7427636540388207
logreg-0002 0.7214276072415384
logreg-0003 0.7413343210679839
logreg-0004 0.7233102020108867
logreg-0005 0.7211794219810611


In [8]:
del train
del test

In [10]:
n_fold = 5
folds = KFold(n_splits=n_fold)

## LightGBM Stacking (bad)

In [11]:


params = {
    'learning_rate': 0.001,
    'num_leaves': 20,
    'max_depth': 1,
    'min_child_weight': 10,
    'lambda_l1':2,
    'lambda_l2':3,
    'min_data_in_leaf' :10,
    'min_sum_hessian_in_leaf' : 0.0001,
    'bagging_fraction' : 0.8,
    'max_bin': 12,
    'feature_fraction' : 0.9,
    'bagging_freq' : 100,
    'min_gain_to_split': 0.1 }

train_options = {
    "model_type":'lgb',
    "params": params,
    "eval_metric":'auc',
    'early_stopping_rounds': 500,
    'n_estimators': 5000,
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
}


result_dict_lgb = train_model_classification(X=oof, X_test=prediction, y=y, params=params, folds=folds,
                                         model_type=train_options['model_type'], 
                                         eval_metric=train_options['eval_metric'],
                                         plot_feature_importance=True,
                                         verbose=500, early_stopping_rounds=train_options['early_stopping_rounds'],
                                         n_estimators=train_options['n_estimators'], 
                                         averaging=train_options['averaging'],
                                         n_jobs=-1, groups=None)

Fold 1 started at Fri Sep  6 22:32:11 2019
Training until validation scores don't improve for 500 rounds.
[500]	training's binary_logloss: 0.109891	training's auc: 0.906504	valid_1's binary_logloss: 0.0839437	valid_1's auc: 0.899767
Early stopping, best iteration is:
[102]	training's binary_logloss: 0.139285	training's auc: 0.906217	valid_1's binary_logloss: 0.102694	valid_1's auc: 0.899856
Fold 2 started at Fri Sep  6 22:33:04 2019
Training until validation scores don't improve for 500 rounds.
[500]	training's binary_logloss: 0.101532	training's auc: 0.903072	valid_1's binary_logloss: 0.1142	valid_1's auc: 0.898168


KeyboardInterrupt: 

## Keras Stacking 

In [17]:
def StackModel_maker():
    k.clear_session()
    
#     categorical_inputs = []
#     for cat in categorical:
#         categorical_inputs.append(Input(shape=[1], name=cat))

#     categorical_embeddings = []
#     for i, cat in enumerate(categorical):
#         categorical_embeddings.append(
#             Embedding(category_counts[cat], int(np.log1p(category_counts[cat]) + 1), name = cat + \
#                       "_embed")(categorical_inputs[i]))

#     categorical_logits = Concatenate(name = "categorical_conc")([Flatten()(SpatialDropout1D(.1)(cat_emb)) for cat_emb in categorical_embeddings])
# 
    numerical_inputs = Input(shape=[oof.shape[1]], name = 'all')
    numerical_logits = Dropout(.3)(numerical_inputs)
  
    x = numerical_logits

    x = Dense(50, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(10, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = BatchNormalization()(x)    
    
    out = Dense(1, activation = 'sigmoid')(x)    

    model = Model(inputs= [numerical_inputs],outputs=out)
    loss = "binary_crossentropy"
    model.compile(optimizer=Adam(lr = 0.0003), loss = loss)
    return model


params = {
    'batch_size': 16384,
    'epochs': 120,
    'verbose': True,
         }
train_options = {
    "model_type":'keras',
    "params": params,
    "eval_metric":'auc',
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
   
}

In [18]:
with open(f'{model_folder}/training_params.json', 'w') as f:
    q = json.dumps(train_options,indent=2)
    f.write(q)
# StackModel_maker().save(f'{model_folder}/keras.mdl')

In [19]:
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU':4}, log_device_placement=False) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [20]:
gc.collect()
result_dict_keras = train_model_classification(model=StackModel_maker, 
                                             X=oof,
                                             X_test=prediction,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             averaging=train_options['averaging'],
                                             groups=None)

Fold 1 started at Fri Sep  6 23:36:48 2019
Train on 472432 samples, validate on 118108 samples
Epoch 1/120
472432/472432 [==============================] - ETA: 13s - loss: 0.76 - ETA: 3s - loss: 0.7664 - ETA: 1s - loss: 0.764 - ETA: 1s - loss: 0.759 - ETA: 0s - loss: 0.754 - ETA: 0s - loss: 0.750 - ETA: 0s - loss: 0.745 - ETA: 0s - loss: 0.742 - ETA: 0s - loss: 0.738 - ETA: 0s - loss: 0.734 - 1s 3us/step - loss: 0.7338 - val_loss: 0.6349
Epoch 2/120
472432/472432 [==============================] - ETA: 0s - loss: 0.704 - ETA: 0s - loss: 0.696 - ETA: 0s - loss: 0.694 - ETA: 0s - loss: 0.691 - ETA: 0s - loss: 0.687 - ETA: 0s - loss: 0.684 - ETA: 0s - loss: 0.680 - ETA: 0s - loss: 0.677 - ETA: 0s - loss: 0.674 - ETA: 0s - loss: 0.672 - 1s 1us/step - loss: 0.6716 - val_loss: 0.6294
Epoch 3/120
472432/472432 [==============================] - ETA: 0s - loss: 0.642 - ETA: 0s - loss: 0.642 - ETA: 0s - loss: 0.640 - ETA: 0s - loss: 0.638 - ETA: 0s - loss: 0.635 - ETA: 0s - loss: 0.632 - ETA: 

472432/472432 [==============================] - ETA: 0s - loss: 0.256 - ETA: 0s - loss: 0.260 - ETA: 0s - loss: 0.260 - ETA: 0s - loss: 0.261 - ETA: 0s - loss: 0.261 - ETA: 0s - loss: 0.260 - ETA: 0s - loss: 0.259 - ETA: 0s - loss: 0.258 - ETA: 0s - loss: 0.258 - ETA: 0s - loss: 0.258 - 1s 1us/step - loss: 0.2578 - val_loss: 0.2514
Epoch 25/120
472432/472432 [==============================] - ETA: 0s - loss: 0.248 - ETA: 0s - loss: 0.248 - ETA: 0s - loss: 0.250 - ETA: 0s - loss: 0.249 - ETA: 0s - loss: 0.249 - ETA: 0s - loss: 0.248 - ETA: 0s - loss: 0.248 - ETA: 0s - loss: 0.248 - ETA: 0s - loss: 0.247 - ETA: 0s - loss: 0.247 - 1s 1us/step - loss: 0.2473 - val_loss: 0.2401
Epoch 26/120
472432/472432 [==============================] - ETA: 0s - loss: 0.249 - ETA: 0s - loss: 0.239 - ETA: 0s - loss: 0.237 - ETA: 0s - loss: 0.238 - ETA: 0s - loss: 0.237 - ETA: 0s - loss: 0.237 - ETA: 0s - loss: 0.238 - ETA: 0s - loss: 0.237 - ETA: 0s - loss: 0.237 - ETA: 0s - loss: 0.236 - 1s 1us/step - l

472432/472432 [==============================] - ETA: 0s - loss: 0.132 - ETA: 0s - loss: 0.130 - ETA: 0s - loss: 0.130 - ETA: 0s - loss: 0.131 - ETA: 0s - loss: 0.130 - ETA: 0s - loss: 0.130 - ETA: 0s - loss: 0.130 - ETA: 0s - loss: 0.130 - ETA: 0s - loss: 0.129 - ETA: 0s - loss: 0.129 - 1s 1us/step - loss: 0.1293 - val_loss: 0.1091
Epoch 48/120
472432/472432 [==============================] - ETA: 0s - loss: 0.123 - ETA: 0s - loss: 0.127 - ETA: 0s - loss: 0.128 - ETA: 0s - loss: 0.129 - ETA: 0s - loss: 0.129 - ETA: 0s - loss: 0.128 - ETA: 0s - loss: 0.127 - ETA: 0s - loss: 0.128 - ETA: 0s - loss: 0.128 - ETA: 0s - loss: 0.127 - 1s 1us/step - loss: 0.1273 - val_loss: 0.1084
Epoch 49/120
472432/472432 [==============================] - ETA: 0s - loss: 0.122 - ETA: 0s - loss: 0.127 - ETA: 0s - loss: 0.126 - ETA: 0s - loss: 0.126 - ETA: 0s - loss: 0.125 - ETA: 0s - loss: 0.125 - ETA: 0s - loss: 0.126 - ETA: 0s - loss: 0.125 - ETA: 0s - loss: 0.125 - ETA: 0s - loss: 0.125 - 1s 2us/step - l

472432/472432 [==============================] - ETA: 0s - loss: 0.103 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.101 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.101 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.100 - 1s 1us/step - loss: 0.1001 - val_loss: 0.0747
Epoch 71/120
472432/472432 [==============================] - ETA: 0s - loss: 0.096 - ETA: 0s - loss: 0.096 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.099 - 1s 1us/step - loss: 0.0997 - val_loss: 0.0733
Epoch 72/120
472432/472432 [==============================] - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.101 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.098 - 1s 1us/step - l

472432/472432 [==============================] - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.088 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - 1s 1us/step - loss: 0.0912 - val_loss: 0.0645
Epoch 94/120
472432/472432 [==============================] - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - 1s 1us/step - loss: 0.0918 - val_loss: 0.0637
Epoch 95/120
472432/472432 [==============================] - ETA: 0s - loss: 0.093 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - 1s 1us/step - l

...

472432/472432 [==============================] - ETA: 1s - loss: 0.221 - ETA: 0s - loss: 0.212 - ETA: 0s - loss: 0.211 - ETA: 0s - loss: 0.211 - ETA: 0s - loss: 0.210 - ETA: 0s - loss: 0.210 - ETA: 0s - loss: 0.210 - ETA: 0s - loss: 0.209 - ETA: 0s - loss: 0.210 - ETA: 0s - loss: 0.209 - 1s 1us/step - loss: 0.2093 - val_loss: 0.2174
Epoch 28/120
472432/472432 [==============================] - ETA: 0s - loss: 0.201 - ETA: 0s - loss: 0.203 - ETA: 0s - loss: 0.203 - ETA: 0s - loss: 0.202 - ETA: 0s - loss: 0.202 - ETA: 0s - loss: 0.202 - ETA: 0s - loss: 0.202 - ETA: 0s - loss: 0.202 - ETA: 0s - loss: 0.201 - ETA: 0s - loss: 0.201 - 1s 1us/step - loss: 0.2011 - val_loss: 0.2086
Epoch 29/120
472432/472432 [==============================] - ETA: 1s - loss: 0.198 - ETA: 0s - loss: 0.195 - ETA: 0s - loss: 0.196 - ETA: 0s - loss: 0.197 - ETA: 0s - loss: 0.196 - ETA: 0s - loss: 0.196 - ETA: 0s - loss: 0.195 - ETA: 0s - loss: 0.195 - ETA: 0s - loss: 0.194 - ETA: 0s - loss: 0.194 - 1s 1us/step - l

472432/472432 [==============================] - ETA: 0s - loss: 0.113 - ETA: 0s - loss: 0.113 - ETA: 0s - loss: 0.115 - ETA: 0s - loss: 0.115 - ETA: 0s - loss: 0.117 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.117 - ETA: 0s - loss: 0.117 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.117 - 1s 1us/step - loss: 0.1169 - val_loss: 0.1228
Epoch 51/120
472432/472432 [==============================] - ETA: 0s - loss: 0.114 - ETA: 0s - loss: 0.114 - ETA: 0s - loss: 0.115 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.116 - 1s 1us/step - loss: 0.1160 - val_loss: 0.1201
Epoch 52/120
472432/472432 [==============================] - ETA: 0s - loss: 0.106 - ETA: 0s - loss: 0.111 - ETA: 0s - loss: 0.113 - ETA: 0s - loss: 0.113 - ETA: 0s - loss: 0.114 - ETA: 0s - loss: 0.114 - ETA: 0s - loss: 0.114 - ETA: 0s - loss: 0.114 - ETA: 0s - loss: 0.114 - ETA: 0s - loss: 0.114 - 1s 1us/step - l

472432/472432 [==============================] - ETA: 0s - loss: 0.097 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.097 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.097 - 1s 1us/step - loss: 0.0974 - val_loss: 0.0928
Epoch 74/120
472432/472432 [==============================] - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.098 - 1s 1us/step - loss: 0.0978 - val_loss: 0.0902
Epoch 75/120
472432/472432 [==============================] - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.097 - ETA: 0s - loss: 0.097 - ETA: 0s - loss: 0.097 - ETA: 0s - loss: 0.097 - ETA: 0s - loss: 0.097 - ETA: 0s - loss: 0.097 - ETA: 0s - loss: 0.097 - ETA: 0s - loss: 0.097 - 1s 1us/step - l

472432/472432 [==============================] - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - 1s 1us/step - loss: 0.0925 - val_loss: 0.0842
Epoch 97/120
472432/472432 [==============================] - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.093 - ETA: 0s - loss: 0.093 - ETA: 0s - loss: 0.093 - ETA: 0s - loss: 0.093 - ETA: 0s - loss: 0.093 - ETA: 0s - loss: 0.093 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - 1s 1us/step - loss: 0.0923 - val_loss: 0.0841
Epoch 98/120
472432/472432 [==============================] - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.094 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - ETA: 0s - loss: 0.092 - 1s 1us/step - l

472432/472432 [==============================] - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.089 - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.090 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.091 - ETA: 0s - loss: 0.090 - 1s 1us/step - loss: 0.0910 - val_loss: 0.0835
Epoch 120/120
118108/118108 [==============================] - ETA:  - 0s 1us/step
Fold 4. auc: 0.9215.
CV mean score: 0.9403, std: 0.0117.


In [21]:
sub = pd.read_csv(f'../../data/sample_submission.csv')
sub['isFraud'] = result_dict_keras['prediction']
sub.to_csv(f'{model_folder}/stacked.csv', index=False)

In [22]:
import pickle
with open(f'{model_folder}/results_dict.pkl', 'wb') as f:
#     q = json.dumps(result_dict_lgb,indent=2)
    pickle.dump(result_dict_keras,f)
#     f.write(q)